In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from mewpy.cobra.util import add_enzyme_constraints
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene,retreive_protein
import json

In [5]:
model = read_sbml_model("../models/non-ec/iAF1260.xml")
sim = get_simulator(model)
sim.set_objective("BIOMASS")

No objective coefficients in model. Unclear what should be optimized


In [6]:
sim.simulate()

objective: 28.21575880575294
Status: OPTIMAL
Method:SimulationMethod.FBA

### Retreive MW infos

The protein IDs and MW (Da) are obtained from UniProt.

Note: does not retreive Kcats from BRENDA. All catalytic rates are equal to 100000/s 

In [7]:
data_all = dict()
try:
    with open(sim.id+".json", 'r') as f:
        data = json.load(f)
except:
    data = dict()
    for gene in sim.genes:
        try:
            info = retreive_gene(gene,"Escherichia coli")
            # set the inverse kcat to a low value
            data[gene]={'protein':info['protein'],'mw':info['mw']*0.001,'kcat':1e-5}
            data_all[gene]=info
        except:
            data[gene]={'protein': gene[len(sim._g_prefix):], 'mw': 0, 'kcat': 0}
    with open(sim.id+".json", "w") as f:
        json.dump(data,f)

In [8]:
print("Genes with missing data:")
for k,v in data.items():
    if v['mw']==0:
        print(k)

Genes with missing data:
b1692


In [9]:
print(data)

{'b0002': {'protein': 'P00561', 'sequence': 'MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSRARISVVLITQSSSEYSISFCVPQSDCVRAERAMQEEFYLELKEGLLEPLAVTERLAIISVVGDGMRTLRGISAKFFAALARANINIVAIAQGSSERSISVVVNNDDATTGVRVTHQMLFNTDQVIEVFVIGVGGVGGALLEQLKRQQSWLKNKHIDLRVCGVANSKALLTNVHGLNLENWQEELAQAKEPFNLGRLIRLVKEYHLLNPVIVDCTSSQAVADQYADFLREGFHVVTPNKKANTSSMDYYHQLRYAAEKSRRKFLYDTNVGAGLPVIENLQNLLNAGDELMKFSGILSGSLSYIFGKLDEGMSFSEATTLAREMGYTEPDPRDDLSGMDVARKLLILARETGRELELADIEIEPVLPAEFNAEGDVAAFMANLSQLDDLFAARVAKARDEGKVLRYVGNIDEDGVCRVKIAEVDGNDPLFKVKNGENALAFYSHYYQPLPLVLRGYGAGNDVTAAGVFADLLRTLSWKLGV', 'ec': ['1.1.1.3', '1.1.1.3', '2.7.2.4'], 'mw': 89.12, 'kcat': 1e-05}, 'b0003': {'protein': 'P00547', 'sequence': 'MVKVYAPASSANMSVGFD

### Build the ec model

In [10]:
ec_sim = add_enzyme_constraints(sim, data)

Read LP format model from file /var/folders/1q/4qg3t0gx0sn39bk35trp5hmm0000gn/T/tmpln8yoc6q.lp
Reading time = 0.01 seconds
: 1668 rows, 4764 columns, 18462 nonzeros


Converting to irreversible: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2382/2382 [00:01<00:00, 2020.46it/s]


Read LP format model from file /var/folders/1q/4qg3t0gx0sn39bk35trp5hmm0000gn/T/tmpmusyj0ta.lp
Reading time = 0.02 seconds
: 1668 rows, 6478 columns, 22304 nonzeros


Splitting isozymes:   0%|                                                                                                                                                          | 0/3239 [00:00<?, ?it/s]/usr/local/Caskroom/miniforge/base/envs/tmcom/lib/python3.9/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
Adding proteins usage to reactions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6548/6548 [09:08<00:00, 11.94it/s]


### Protein pool usage

In [11]:
res = ec_sim.simulate()
print(res)

objective: 28.21575880575297
Status: OPTIMAL
Method:SimulationMethod.FBA


In [12]:
res.find('pool')

,Flux rate
Reaction ID,
prot_pool_exchange,390.099432


### Save the model

In [13]:
write_sbml_model(ec_sim.model,"eciAF1260.xml")

In [14]:
env = Environment.complete(ec_sim, max_uptake=1000.0, inplace=False)

In [9]:
env['prot_pool_exchange']=(0,0.8)

env: ['prot_pool_exchange']=(0,0.8)


In [8]:
ec_sim.simulate(constraints=env)

NameError: name 'ec_sim' is not defined